<a href="https://colab.research.google.com/github/thilankam/Converting_Tabular_Data_to_Graph/blob/main/Converting_Tabular_Data_to_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Download data (quietly)
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player.csv
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player_skill.csv
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_team.csv

# Load data
player_df = pd.read_csv("tbl_player.csv")
skill_df = pd.read_csv("tbl_player_skill.csv")
team_df = pd.read_csv("tbl_team.csv")

# check the head of the dataframes
player_df.head()
skill_df.head()
team_df.head()

# Obtaining subsets from each dataframe
player_df = player_df[['int_player_id', 'str_player_name', 'str_positions', 'int_team_id']]
player_df.head()
skill_df = skill_df[['int_player_id', 'int_long_passing', 'int_ball_control', 'int_dribbling']]
skill_df.head()
team_df = team_df[['int_team_id','str_team_name','int_overall']]
team_df.head()

# Merging the datasets
player_df = player_df.merge(skill_df, on = 'int_player_id')
player_df.head()
fifa_df = player_df.merge(team_df, on = 'int_team_id')
fifa_df.head()

# Sorting the dataframe based on the overall rating column. 
fifa_df = fifa_df.sort_values(by = 'int_overall', ascending= False)
fifa_df.head()
fifa_df.shape[0]
print("Players: ", fifa_df.shape[0])
fifa_df.head()

Players:  18767


,int_player_id,str_player_name,str_positions,int_team_id,int_long_passing,int_ball_control,int_dribbling,str_team_name,int_overall
181,33,Andrew Robertson,LB,1.0,76,82,80,Liverpool,85
192,507,James Philip Milner,"CM, RB, LB",1.0,81,80,77,Liverpool,85
94,34,Aymeric Laporte,CB,2.0,80,78,63,Manchester City,85
93,32,Bernardo Mota Veiga de Carvalho e Silva,"RW, CAM, CM",2.0,76,92,92,Manchester City,85
92,23,Raheem Shaquille Sterling,"LW, RW",2.0,69,88,90,Manchester City,85


In [ ]:
# Make suree that we don't have duplicate nodes
max(fifa_df['int_player_id'].value_counts())

1

Extracting the node features:

The node features are typically represented in a matrix of the shape (num_nodes, node_feature_dim).

For each of the football players, we simply extract their attributes. Because each player id is unique, we can easily do this based on the original dataframe.

In [35]:
# Sorting to define the order of the nodes
sorted_df = fifa_df.sort_values(by = 'int_player_id')
# Select node features
node_features = sorted_df[['str_positions', 'int_long_passing', 'int_ball_control', 'int_dribbling']]
# convert the non-numeric colums
pd.set_option('mode.chained_assignment', None)
positions = node_features['str_positions'].str.split(",", expand = True)
node_features['first_position'] = positions[0]
# one-hot encoding
node_features = pd.concat([node_features, pd.get_dummies(node_features['first_position'])], axis=1, join='inner')
node_features.drop(['str_positions', 'first_position'], axis=1, inplace=True)
node_features.head()

,int_long_passing,int_ball_control,int_dribbling,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,91,96,96,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
33,77,92,88,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
57,40,30,12,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
89,93,92,88,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
121,81,95,95,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


That's already our node feature matrix. The number of nodes and the ordering is implicitly defined by it's shape. Each row corresponds to one node in our final graph.

In [ ]:
# Convert to numpy
